In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


In [27]:
user_df = pd.read_csv('/content/users.csv')
user_df.head()

,login,name,company,location,email,hireable,bio,public_repos,followers,following,created_at
0,orta,Orta Therox,NaN,Huddersfield / NYC / Dublin / Rio de Janeiro,git@orta.io,NaN,Tech on @puzzmo-com \r\n\r\nEx-TypeScript. Con...,952,6016,109,2009-01-24T20:40:31Z
1,jeromeetienne,Jerome Etienne,MAKING WEBAR A REALITY,"Dublin, Ireland",jerome.etienne@gmail.com,True,Making WebAR a reality - Around Javascript and...,301,2779,15,2010-04-26T11:58:29Z
2,jonataslaw,Jonny Borges,IRIS,"Dublin, Ireland",NaN,True,VP of Engineering from Iris Finance.\r\nDevelo...,272,2686,17,2018-01-23T19:17:15Z
3,steventroughtonsmith,Steven Troughton-Smith,HIGH CAFFEINE CONTENT,"Dublin, Ireland",NaN,NaN,NaN,98,1982,4,2009-01-08T23:51:31Z
4,axic,Alex Beregszaszi,ETHEREUM @IPSILON @SPEARBIT @ETHEREUMJS,"Dublin, Ireland",NaN,True,Works on decentralised stuff.\r\n\r\neipnft:ax...,143,1802,5,2008-08-11T23:38:10Z


In [3]:
user_df.columns

Index(['login', 'name', 'company', 'location', 'email', 'hireable', 'bio',
       'public_repos', 'followers', 'following', 'created_at'],
      dtype='object')

In [5]:
user_df.sort_values(by='followers', ascending=False).head(5)['login']

,login
0,orta
1,jeromeetienne
2,jonataslaw
3,steventroughtonsmith
4,axic


In [11]:
user_df.sort_values(by='created_at', ascending=True).head(5)['login']

,login
75,paulca
445,adrian
85,GavinJoyce
74,amir
439,ciaranlee


In [12]:
repo_df=pd.read_csv('/content/repositories.csv')

In [13]:
repo_df.columns

Index(['login', 'full_name', 'created_at', 'stargazers_count',
       'watchers_count', 'language', 'has_projects', 'has_wiki',
       'license_name'],
      dtype='object')

In [17]:

license_counts = repo_df['license_name'].value_counts()
top_3_licenses = license_counts.head(3).index.tolist()
print(','.join(top_3_licenses))

mit,apache-2.0,other


In [18]:

company_counts = user_df['company'].value_counts()
if not company_counts.empty:
  most_common_company = company_counts.index[0]
  print(most_common_company)
else:
  print("No company information available.")

AWS


In [19]:
# prompt: Which programming language is most popular among these users?

language_counts = repo_df['language'].value_counts()
if not language_counts.empty:
  most_popular_language = language_counts.index[0]
  print(most_popular_language)
else:
  print("No language information available.")

JavaScript


In [21]:
# prompt: Which programming language is the second most popular among users who joined after 2020?

# Convert 'created_at' to datetime objects
user_df['created_at'] = pd.to_datetime(user_df['created_at'])

# Filter users who joined after 2020
recent_users = user_df[user_df['created_at'].dt.year > 2020]

# Get language counts for recent users
language_counts_recent = repo_df[repo_df['login'].isin(recent_users['login'])]['language'].value_counts()

if not language_counts_recent.empty:
  second_most_popular_language = language_counts_recent.index[1] if len(language_counts_recent) > 1 else "Only one language found"
  print(second_most_popular_language)
else:
  print("No language information available for users who joined after 2020.")

Jupyter Notebook


In [22]:
# prompt: Which language has the highest average number of stars per repository?

# Group by language and calculate the average number of stars
average_stars_by_language = repo_df.groupby('language')['stargazers_count'].mean()

# Find the language with the highest average number of stars
if not average_stars_by_language.empty:
  language_with_highest_average_stars = average_stars_by_language.idxmax()
  print(language_with_highest_average_stars)
else:
  print("No language information available.")

MDX


In [23]:
# prompt: Let's define leader_strength as followers / (1 + following). Who are the top 5 in terms of leader_strength? List their login in order, comma-separated.

# Calculate leader_strength
user_df['leader_strength'] = user_df['followers'] / (1 + user_df['following'])

# Sort by leader_strength in descending order and get the top 5
top_5_leaders = user_df.sort_values('leader_strength', ascending=False).head(5)

# Extract the logins and join them with commas
top_5_logins = ','.join(top_5_leaders['login'].tolist())

top_5_logins

'flaviohenriquealmeida,zalando,AnikSarker,wix,CardinalHealth'

In [24]:
# prompt: What is the correlation between the number of followers and the number of public repositories among users in Dublin?

# Assuming 'user_df' and 'repo_df' are already loaded as DataFrames

# Filter users from Dublin (replace 'Dublin' with the actual location identifier if different)
dublin_users = user_df[user_df['location'] == 'Dublin']

# Merge user data with repository data based on 'login'
dublin_user_repo_data = pd.merge(dublin_users, repo_df, on='login', how='inner')

# Calculate the correlation between followers and public repository count
correlation = dublin_user_repo_data['followers'].corr(dublin_user_repo_data['login'].map(dublin_user_repo_data.groupby('login')['name'].count()))

print(f"Correlation between followers and public repositories for Dublin users: {correlation}")

Correlation between followers and public repositories for Dublin users: 0.28273434143888315


In [28]:
# prompt: Does creating more repos help users get more followers? Using regression, estimate how many additional followers a user gets per additional public repository.

# Merge user data with repository data based on 'login'
user_repo_data = pd.merge(user_df, repo_df, on='login', how='inner')

# Group by user and count the number of repositories
repo_counts = user_repo_data.groupby('login')['name'].count().reset_index(name='repo_count')

# Merge the repository counts back into the user dataframe
user_df = pd.merge(user_df, repo_counts, on='login', how='left')

# Fill NaN values in repo_count with 0 (users with no repositories)
user_df['repo_count'] = user_df['repo_count'].fillna(0)

# Perform linear regression to estimate the relationship between followers and repo_count
from sklearn.linear_model import LinearRegression

X = user_df['repo_count'].values.reshape(-1, 1)  # Independent variable (repo count)
y = user_df['followers'].values  # Dependent variable (followers)

model = LinearRegression()
model.fit(X, y)

# Get the slope of the regression line (additional followers per additional repo)
followers_per_repo = model.coef_[0]

print(f"Estimated additional followers per public repository: {followers_per_repo:.3f}")

Estimated additional followers per public repository: 1.985


In [31]:
# prompt: Do people typically enable projects and wikis together? What is the correlation between a repo having projects enabled and having wiki enabled?

# Assuming 'repo_df' is already loaded as a DataFrame

# Check if 'has_projects' and 'has_wiki' columns exist in the DataFrame
if 'has_projects' in repo_df.columns and 'has_wiki' in repo_df.columns:
  # Calculate the correlation between 'has_projects' and 'has_wiki'
  correlation = repo_df['has_projects'].corr(repo_df['has_wiki'])

  print(f"Correlation between having projects enabled and having wiki enabled: {correlation:.3f}")

  # Calculate the percentage of repositories with both projects and wiki enabled
  both_enabled = repo_df[(repo_df['has_projects'] == True) & (repo_df['has_wiki'] == True)].shape[0]
  total_repos = repo_df.shape[0]
  percentage_both_enabled = (both_enabled / total_repos) * 100

  print(f"Percentage of repositories with both projects and wiki enabled: {percentage_both_enabled:.3f}%")
else:
  print("Columns 'has_projects' and/or 'has_wiki' not found in the DataFrame.")

Correlation between having projects enabled and having wiki enabled: 0.334
Percentage of repositories with both projects and wiki enabled: 85.340%


In [37]:
# prompt: Do hireable users follow more people than those who are not hireable?
# Average of following per user for hireable=true minus the average following for the rest

# Calculate the average following for hireable users
hireable_following_avg = user_df[user_df['hireable'] == True]['following'].mean()

# Calculate the average following for non-hireable users
non_hireable_following_avg = user_df[user_df['hireable'] != True]['following'].mean()

# Calculate the difference
difference = hireable_following_avg - non_hireable_following_avg

print(f"Difference in average following between hireable and non-hireable users: {difference:.3f}")

Difference in average following between hireable and non-hireable users: 47.707


In [38]:
# prompt: Some developers write long bios. Does that help them get more followers? What's the correlation of the length of their bio (in Unicode words, split by whitespace) with followers? (Ignore people without bios)

# Assuming 'user_df' is already loaded as a DataFrame

# Remove rows with missing bios
user_df_with_bios = user_df.dropna(subset=['bio'])

# Calculate bio length in words
user_df_with_bios['bio_length'] = user_df_with_bios['bio'].apply(lambda x: len(str(x).split()))

# Calculate the correlation between bio length and followers
correlation = user_df_with_bios['bio_length'].corr(user_df_with_bios['followers'])

print(f"Correlation between bio length and followers: {correlation:.3f}")

Correlation between bio length and followers: 0.116


<ipython-input-38-a80dee9e73c6>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  user_df_with_bios['bio_length'] = user_df_with_bios['bio'].apply(lambda x: len(str(x).split()))


In [43]:
user_repo_data.columns

Index(['login', 'name', 'company', 'location', 'email', 'hireable', 'bio',
       'public_repos', 'followers', 'following', 'created_at_x', 'full_name',
       'created_at_y', 'stargazers_count', 'watchers_count', 'language',
       'has_projects', 'has_wiki', 'license_name'],
      dtype='object')

In [45]:
# prompt: Who created the most repositories on weekends (UTC)? List the top 5 users' login in order, comma-separated

# Convert 'created_at' column to datetime objects
repo_df['created_at'] = pd.to_datetime(repo_df['created_at'])

# Extract the day of the week (0=Monday, 6=Sunday)
repo_df['day_of_week'] = repo_df['created_at'].dt.dayofweek

# Filter for weekend repositories (Saturday and Sunday)
weekend_repos = repo_df[(repo_df['day_of_week'] == 5) | (repo_df['day_of_week'] == 6)]

# Group by user login and count the number of weekend repositories
weekend_repo_counts = weekend_repos.groupby('login')['login'].count().reset_index(name='weekend_repo_count')

# Sort the users by the number of weekend repositories in descending order
top_users = weekend_repo_counts.sort_values('weekend_repo_count', ascending=False).head(5)

# Extract the logins and join them with commas
top_5_logins = ','.join(top_users['login'].tolist())

top_5_logins


'joshuacassidy,skilbjo,kracekumar,butlerx,aldanor'

In [41]:
# prompt: Do people who are hireable share their email addresses more often?
# [fraction of users with email when hireable=true] minus [fraction of users with email for the rest] (to 3 decimal places, e.g. 0.123 or -0.123)

# Assuming 'user_df' is already loaded as a DataFrame

# Calculate the fraction of hireable users with email
hireable_with_email = user_df[(user_df['hireable'] == True) & (user_df['email'].notna())].shape[0]
total_hireable = user_df[user_df['hireable'] == True].shape[0]
fraction_hireable_with_email = hireable_with_email / total_hireable if total_hireable > 0 else 0

# Calculate the fraction of non-hireable users with email
non_hireable_with_email = user_df[(user_df['hireable'] != True) & (user_df['email'].notna())].shape[0]
total_non_hireable = user_df[user_df['hireable'] != True].shape[0]
fraction_non_hireable_with_email = non_hireable_with_email / total_non_hireable if total_non_hireable > 0 else 0

# Calculate the difference
difference = fraction_hireable_with_email - fraction_non_hireable_with_email

print(f"{difference:.3f}")

0.108


In [47]:
# prompt:  Let's assume that the last word in a user's name is their surname (ignore missing names, trim and split by whitespace.) What's the most common surname? (If there's a tie, list them all, comma-separated, alphabetically)

# Assuming 'user_df' is already loaded as a DataFrame

# Extract surnames (last word in the 'name' column)
def get_surname(name):
    if isinstance(name, str):
        parts = name.strip().split()
        if parts:
            return parts[-1]
    return None

user_df['surname'] = user_df['name'].apply(get_surname)

# Count surname occurrences
surname_counts = user_df['surname'].value_counts()
print(surname_counts)
# Find the most common surname(s)
max_count = surname_counts.max()
most_common_surnames = surname_counts[surname_counts == max_count].index.tolist()

# Sort and join surnames if there's a tie
most_common_surnames.sort()
print(','.join(most_common_surnames))

surname
O'Sullivan       3
Kenny            3
Chen             3
Quinn            3
Lee              2
                ..
Suriyawongkul    1
Aboltins         1
Rodrigues        1
Finassi          1
McCool           1
Name: count, Length: 445, dtype: int64
Chen,Kenny,O'Sullivan,Quinn
